In [1]:
from datasets import load_dataset

In [46]:

import os
import openai
import json
import numpy as np
from tqdm import tqdm
import backoff 
openai.api_key = json.load(open("/Users/shahules/openai-key.json"))['ikka']

In [65]:

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def llm(prompt, **kwargs):
    response = openai.ChatCompletion.create(
        model=kwargs.get("model", "gpt-4"),
        messages=[{"role":"user","content":prompt}],
        temperature=kwargs.get("temperature", 0),
        top_p=kwargs.get("top_p", 1),
        frequency_penalty=kwargs.get("frequency_penalty", 0.0),
        presence_penalty=kwargs.get("presence_penalty", 0.0),
        max_tokens=kwargs.get("max_tokens", 500),
        n=kwargs.get("n", 1),
    )
    return response['choices'][0]['message']['content']


## Prepare ELI5 Data

In [2]:
eli5_dataset = load_dataset("Pakulski/ELI5-test")

Extracting data files: 100%|█████████████████████████████| 1/1 [00:00<00:00, 106.41it/s]


Dataset parquet downloaded and prepared to /Users/shahules/.cache/huggingface/datasets/Pakulski___parquet/Pakulski--ELI5-test-ed159b4d22db0b30/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.83it/s]


In [3]:
eli5_dataset = eli5_dataset.filter(lambda ex: ex["question"].endswith("?"))

In [104]:
def prepara_prompt(example):
    
    prompt = """
    Answer the follwing question using information in the given context. 
    If information is the given context is not enough to answer the question reply 'insufficient information'
    question:{question}
    context:\n{context}
    answer:
    """
    
    q, d = example['question'], example['document']
    return prompt.format(question=q,context=d)
    
    
def extend_context(example):
    
    prompt = """
    Add one extra paragraph containing 3 to 4 sentences to given text.
    context:{context}
    """
    docs = example['contexts'][:]
    idx = np.random.randint(0,len(docs))
    chunk = llm(prompt.format(context=docs[idx]))
    docs.insert(idx+1, chunk)
    return docs
    
    
def less_relevant_answer(example):
    
    prompt =  """
    Answer the given question partially.
    question:{question}
    answer:
    """
    q = example['question']
    prompt = prompt.format(question=q)
    return llm(prompt)
    
def get_ungrounded_answer(example):
    
    prompt = """
    Answer the following question without being too long. 
    question:{question}
    answer:
    """
    q = example['question']
    prompt = prompt.format(question=q)
    return llm(prompt)
    
    
     
    

In [56]:
eli5_ragas = pickle.load(open("eli5_ragas.pkl","rb"))
selected = eli5_dataset['train'].shuffle().select(range(60,100))
for example in tqdm(selected):
    prompt = prepara_prompt(example)
    output = llm(prompt)
    answer = output['choices'][0]['message']['content']
    sample = {"question":example['question'], "contexts":example['document'].split("\n"), 
             "answer":answer, 'id':example['id']}
    eli5_ragas.append(sample)
    


100%|███████████████████████████████████████████████████| 40/40 [07:17<00:00, 10.94s/it]


In [67]:
import pickle
with open("eli5_ragas.pkl", "wb") as obj:
    pickle.dump(eli5_ragas, obj)

In [106]:
for item in tqdm(eli5_ragas[:3]):
    item["poor_answer"] = less_relevant_answer(item)
    item["ungrounded_answer"] = get_ungrounded_answer(item)
    item["context_v2"] = extend_context(item)
    
    

100%|█████████████████████████████████████████████████████| 3/3 [01:08<00:00, 22.99s/it]


In [110]:
item['question']

'Did the Soviets innervate or did they just steal western designs?'

In [109]:
set(item['context_v2']) - set(item['contexts'])

{"The Soviets' approach to missile placement was not only strategic but also indicative of their broader military philosophy. They prioritized speed and firepower, often at the expense of size and durability. This approach was a stark contrast to Western military strategies, which often favored larger, more robust vessels. The Soviets' unique approach to warfare, combined with their meticulous attention to detail in other areas such as coin examination, paints a picture of a nation that was both innovative and pragmatic."}

In [108]:
len(eli5_ragas[0]['contexts'])

9

In [38]:
# eli5_dataset['train'].shuffle().select(range(0,50))

In [57]:
len([item for item in eli5_ragas if item['answer']!='Insufficient information'])

66

In [96]:
x = [1,2,3]

In [97]:
x.insert(1,5)

In [98]:
x

[1, 5, 2, 3]

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)
from ragas import evaluate
from ragas.metrics.context_relevance import ContextRelevancy

In [ ]:
results = evaluate(eli5_dataset, metrics=[faithfulness])